<a href="https://colab.research.google.com/github/akh-04/Amazon_Web_Scraper/blob/main/Selenium_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WEB SCRAPING NOTEBOOK USING BEAUTIFUL SOUP AND SELENIUM
In the notebook, Selenium is used to automate the process of navigating to the Amazon website, searching for a product, and scraping the product details. Beautiful Soup is then used to parse the HTML content of the scraped page and extract the relevant information, such as the product name, price, and rating.


In [ ]:
!pip install selenium                                 # Install selenium
!apt-get update                                       # To update ubuntu to correctly run apt install
!apt install chromium-chromedriver -y                 # Install chrome driver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin   # Adding the driver path

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,014 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
#Creating a Template URL
def get_url(search_term):
    """ Generate a URL for the search term"""
    template="https://www.amazon.com/s?k={}&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent"
    search_term=search_term.replace(' ','+')
    return template.format(search_term)

## Results for individual lines of code which are used later stored in a CSV


Searching for Articles example "keyboard"

In [ ]:
url=get_url('keyboard')
print(url)

https://www.amazon.com/s?k=keyboard&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent


In [ ]:
driver.get(url)

In [ ]:
# Reading The HTML websit
soup=BeautifulSoup(driver.page_source,'html.parser')
results=soup.find_all('div',{'data-component-type':'s-search-result'})
len(results)

16

In [ ]:
# Extracting the description
item=results[0]
item.h2.a.text.strip()

'Logitech G213 Prodigy Gaming Keyboard, LIGHTSYNC RGB Backlit Keys, Spill-Resistant, Customizable Keys, Dedicated Multi-Media Keys – Black'

In [ ]:
# Price of the item
price_parent=item.find('span','a-price')
price=price_parent.find('span','a-price-whole').text.strip()
price

'46.'

In [ ]:
def get_url(search_term):
    """ Generate a URL for the search term"""
    template="https://www.amazon.com/s?k={}&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent"
    search_term=search_term.replace(' ','+')

    #add term query to url
    url=template.format(search_term)

    #add page query place holder
    url += '&page{}'

    return url

In [ ]:
def extract_record(item):
    # Extract description and URL
    atag = item.h2.a
    description = atag.text.strip()  # Get the text of the 'a' tag (product description)
    url = "https://www.amazon.com" + atag.get('href')  # Construct the complete URL for the product

    try:
        # Extract price
        price_parent = item.find('span', 'a-price')  # Find the parent element containing the price
        price = price_parent.find('span', 'a-price-whole').text.strip()  # Get the price text
    except AttributeError:
        # If price information is not available, return None
        return None

    try:
        # Extract rating and review count
        rating = item.i.text  # Get the rating text
        review_count = item.find('span', {'class': 'a-size-base'}).text  # Get the review count text
    except AttributeError:
        # If rating or review count information is not available, set them to empty strings
        rating = ''
        review_count = ''

    # Create a tuple containing the extracted data
    result = (description, price, rating, review_count, url)

    return result


In [ ]:

def search_query(search_term):
    """Run main program routine"""

    # Start up the webdriver
    # This code is for google chrome driver if running else where then change this Chrome driver path with your own path,

    driver = webdriver.Chrome(options=chrome_options)

    records = []  # Create an empty list to store the extracted data
    url = get_url(search_term)  # Get the initial URL for the search term

    # Loop through 20 pages of search results
    for page in range(1, 21):
        driver.get(url.format(page))  # Load the page in the webdriver
        soup = BeautifulSoup(driver.page_source, 'html.parser')  # Parse the HTML content of the page
        results = soup.find_all('div', {'data-component-type': 's-search-result'})  # Find all search result items

        # Loop through each search result item and extract data
        for item in results:
            record = extract_record(item)  # Extract data from the current search result item
            if record:
                records.append(extract_record(item))  # Append the extracted data to the records list

    driver.close()  # Close the webdriver

    df = pd.DataFrame(records, columns=['Description', 'Price(Rs)', 'Rating', 'Review Count', 'Url of the Product'])

    # Display the DataFrame as a table in the console
    display(df)

    filename = search_term + ".csv"  # Create a CSV file name based on the search term

    # Save the data to a CSV file
    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)  # Create a CSV writer
        writer.writerow(['Description', 'Price(Rs)', 'Rating', 'Review Count', 'Url of the Product'])  # Write header row
        writer.writerows(records)  # Write the extracted records to the CSV file

In [ ]:
search_query('Keyboard')

,Description,Price(Rs),Rating,Review Count,Url of the Product
0,Amazon Basics Low-Profile Wired USB Keyboard w...,17.,4.5 out of 5 stars,Amazon's Choice: Overall Pick,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,"TMKB 60 Percent Keyboard,Gaming Keyboard 60 Pe...",27.,4.6 out of 5 stars,4.6,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2,"Gaming Keyboard, 104 Keys All-Metal Panel, Rai...",26.,4.5 out of 5 stars,4.5,https://www.amazon.com/All-Metal-Dacoity-Multi...
3,Rii RK907 Ultra-Slim Compact USB Wired Keyboar...,8.,4.4 out of 5 stars,4.4,https://www.amazon.com/Rii-Ultra-Slim-Compact-...
4,"NPET K10 Wired Gaming Keyboard, LED Backlit, S...",19.,4.4 out of 5 stars,4.4,https://www.amazon.com/NPET-Floating-Keyboard-...
5,Logitech MK270 Wireless Keyboard And Mouse Com...,27.,4.5 out of 5 stars,4.5,https://www.amazon.com/Logitech-MK270-Wireless...
6,Amazon Basics USB Wired Computer Keyboard (QWE...,14.,4.5 out of 5 stars,4.5,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
7,Amazon Basics Low-Profile Wired USB Keyboard w...,17.,4.5 out of 5 stars,Amazon's Choice: Overall Pick,https://www.amazon.com/AmazonBasics-Matte-Keyb...
8,"CHONCHOW LED Keyboard and Mouse, 104 Keys Rain...",19.,4.4 out of 5 stars,4.4,https://www.amazon.com/CHONCHOW-Keyboard-Rainb...
9,"Wireless Keyboard and Mouse Ultra Slim Combo, ...",29.,4.3 out of 5 stars,4.3,https://www.amazon.com/TopMate-Wireless-Keyboa...


##References

1. [Selenium Documentation ](https://selenium-python.readthedocs.io/index.html)

2. [Web Scraping Example ](https://medium.com/ymedialabs-innovation/web-scraping-using-beautiful-soup-and-selenium-for-dynamic-page-2f8ad15efe25)
